In [1]:
import json
import pandas as pd
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import pandas as pd
import pyarrow as pa
import pyarrow.csv as pv

In [2]:
mta= pd.read_csv('MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv')

Got the csv file from here but is only from febuary because they did it differently in january which is a pain here is link

https://data.ny.gov/Transportation/MTA-Subway-Hourly-Ridership-Beginning-February-202/wujg-7c2s

In [3]:
mta.shape

(5015198, 11)

In [4]:
mta.columns

Index(['transit_timestamp', 'station_complex_id', 'station_complex', 'borough',
       'routes', 'payment_method', 'ridership', 'transfers', 'latitude',
       'longitude', 'Georeference'],
      dtype='object')

In [5]:
mta.head

<bound method NDFrame.head of               transit_timestamp station_complex_id  station_complex borough  \
0        06/18/2023 07:00:00 PM               H023    DeKalb Av (L)      BK   
1        06/18/2023 11:00:00 AM               C014  Prospect Av (R)      BK   
2        01/24/2023 10:00:00 PM               R252       103 St (6)       M   
3        09/05/2022 11:00:00 AM               R252       103 St (6)       M   
4        05/08/2022 10:00:00 PM               R170       103 St (1)       M   
...                         ...                ...              ...     ...   
5015193  03/15/2023 05:00:00 PM               R419    Zerega Av (6)      BX   
5015194  04/25/2023 12:00:00 AM               R419    Zerega Av (6)      BX   
5015195  05/12/2023 01:00:00 PM               R419    Zerega Av (6)      BX   
5015196  06/04/2023 05:00:00 AM               R419    Zerega Av (6)      BX   
5015197  04/23/2023 04:00:00 AM               R419    Zerega Av (6)      BX   

        routes paymen

### Checking for logical test so just duplicates and null values.Along with ensuring borough will be same kind of thing like all lowercase

In [6]:
# Checking for duplicates
num_duplicates = mta.duplicated().sum()
print("Number of duplicates:", num_duplicates)

# Checking for null values
num_null_values = mta.isnull().sum().sum()
print("Number of null values:", num_null_values)


Number of duplicates: 0
Number of null values: 0


In [7]:
#Standarise the data just to ensure its ok and all are same 
mta['borough'] = mta['borough'].str.lower()

In [8]:
columns_to_drop = ['payment_method', 'ridership', 'transfers','Georeference','station_complex_id']

df = mta.drop(columns=columns_to_drop)


In [9]:
df.head

<bound method NDFrame.head of               transit_timestamp  station_complex borough routes   latitude  \
0        06/18/2023 07:00:00 PM    DeKalb Av (L)      bk      L  40.703810   
1        06/18/2023 11:00:00 AM  Prospect Av (R)      bk      R  40.665413   
2        01/24/2023 10:00:00 PM       103 St (6)       m      6  40.790600   
3        09/05/2022 11:00:00 AM       103 St (6)       m      6  40.790600   
4        05/08/2022 10:00:00 PM       103 St (1)       m      1  40.799446   
...                         ...              ...     ...    ...        ...   
5015193  03/15/2023 05:00:00 PM    Zerega Av (6)      bx      6  40.836487   
5015194  04/25/2023 12:00:00 AM    Zerega Av (6)      bx      6  40.836487   
5015195  05/12/2023 01:00:00 PM    Zerega Av (6)      bx      6  40.836487   
5015196  06/04/2023 05:00:00 AM    Zerega Av (6)      bx      6  40.836487   
5015197  04/23/2023 04:00:00 AM    Zerega Av (6)      bx      6  40.836487   

         longitude  
0       -73.

In [10]:
df['borough'].unique

<bound method Series.unique of 0          bk
1          bk
2           m
3           m
4           m
           ..
5015193    bx
5015194    bx
5015195    bx
5015196    bx
5015197    bx
Name: borough, Length: 5015198, dtype: object>

I only wanted borough of Manhattan so just to see what manhattan borough would be so filter it to that. 

In [11]:
# Assuming 'df' is the DataFrame you are working with
df = df[df['borough'] == 'm'].drop('borough', axis=1)

In [12]:
df.head

<bound method NDFrame.head of               transit_timestamp                          station_complex  \
2        01/24/2023 10:00:00 PM                               103 St (6)   
3        09/05/2022 11:00:00 AM                               103 St (6)   
4        05/08/2022 10:00:00 PM                               103 St (1)   
5        05/03/2022 10:00:00 PM                                 1 Av (L)   
6        01/28/2023 06:00:00 PM                                 1 Av (L)   
...                         ...                                      ...   
4880452  10/29/2022 06:00:00 AM  West 4 St-Washington Sq (A,B,C,D,E,F,M)   
4880453  01/27/2023 10:00:00 AM                            Wall St (4,5)   
4880455  07/15/2022 05:00:00 AM                            Wall St (4,5)   
4880456  02/27/2023 06:00:00 AM  West 4 St-Washington Sq (A,B,C,D,E,F,M)   
4880458  11/30/2022 08:00:00 PM                            Wall St (4,5)   

                routes   latitude  longitude  
2         

In [ ]:
# Assuming 'df' is the DataFrame you are working with
df['coordinates'] = df[['latitude', 'longitude']].astype(str).apply(lambda x: ' '.join(x), axis=1)

# Get unique coordinates
unique_coordinates = df[['latitude', 'longitude']].drop_duplicates()

In [16]:
df.shape

(1435991, 5)

In [17]:
df.head

<bound method NDFrame.head of               transit_timestamp                          station_complex  \
2        01/24/2023 10:00:00 PM                               103 St (6)   
3        09/05/2022 11:00:00 AM                               103 St (6)   
4        05/08/2022 10:00:00 PM                               103 St (1)   
5        05/03/2022 10:00:00 PM                                 1 Av (L)   
6        01/28/2023 06:00:00 PM                                 1 Av (L)   
...                         ...                                      ...   
4880452  10/29/2022 06:00:00 AM  West 4 St-Washington Sq (A,B,C,D,E,F,M)   
4880453  01/27/2023 10:00:00 AM                            Wall St (4,5)   
4880455  07/15/2022 05:00:00 AM                            Wall St (4,5)   
4880456  02/27/2023 06:00:00 AM  West 4 St-Washington Sq (A,B,C,D,E,F,M)   
4880458  11/30/2022 08:00:00 PM                            Wall St (4,5)   

                routes   latitude  longitude  
2         

In [18]:
df.columns

Index(['transit_timestamp', 'station_complex', 'routes', 'latitude',
       'longitude'],
      dtype='object')

In [19]:
df['transit_timestamp'].head

<bound method NDFrame.head of 2          01/24/2023 10:00:00 PM
3          09/05/2022 11:00:00 AM
4          05/08/2022 10:00:00 PM
5          05/03/2022 10:00:00 PM
6          01/28/2023 06:00:00 PM
                    ...          
4880452    10/29/2022 06:00:00 AM
4880453    01/27/2023 10:00:00 AM
4880455    07/15/2022 05:00:00 AM
4880456    02/27/2023 06:00:00 AM
4880458    11/30/2022 08:00:00 PM
Name: transit_timestamp, Length: 1435991, dtype: object>

In [20]:
df['station_complex'].head

<bound method NDFrame.head of 2                                       103 St (6)
3                                       103 St (6)
4                                       103 St (1)
5                                         1 Av (L)
6                                         1 Av (L)
                            ...                   
4880452    West 4 St-Washington Sq (A,B,C,D,E,F,M)
4880453                              Wall St (4,5)
4880455                              Wall St (4,5)
4880456    West 4 St-Washington Sq (A,B,C,D,E,F,M)
4880458                              Wall St (4,5)
Name: station_complex, Length: 1435991, dtype: object>

In [21]:
df['routes'].head

<bound method NDFrame.head of 2                      6
3                      6
4                      1
5                      L
6                      L
               ...      
4880452    F,M,C,E,B,A,D
4880453              5,4
4880455              5,4
4880456    F,M,C,E,B,A,D
4880458              5,4
Name: routes, Length: 1435991, dtype: object>

Ok all the routes are together so was hard to actually visualise it or see it properly so i used dummies to extract the route information 

In [22]:
# Create dummy variables for each unique value in the 'routes' column
dummy_routes = df['routes'].str.get_dummies(sep=',')
# Get a list of the dummy variable column names
dummy_variables = dummy_routes.columns.tolist()

# Print the list of dummy variable names
print(dummy_variables)
# Concatenate the dummy variables with the original DataFrame
df = pd.concat([df, dummy_routes], axis=1)


['1', '2', '3', '4', '5', '6', '7', 'A', 'B', 'C', 'D', 'E', 'F', 'J', 'L', 'M', 'N', 'Q', 'R', 'S', 'W', 'Z']


In [23]:
print(df)

              transit_timestamp                          station_complex  \
2        01/24/2023 10:00:00 PM                               103 St (6)   
3        09/05/2022 11:00:00 AM                               103 St (6)   
4        05/08/2022 10:00:00 PM                               103 St (1)   
5        05/03/2022 10:00:00 PM                                 1 Av (L)   
6        01/28/2023 06:00:00 PM                                 1 Av (L)   
...                         ...                                      ...   
4880452  10/29/2022 06:00:00 AM  West 4 St-Washington Sq (A,B,C,D,E,F,M)   
4880453  01/27/2023 10:00:00 AM                            Wall St (4,5)   
4880455  07/15/2022 05:00:00 AM                            Wall St (4,5)   
4880456  02/27/2023 06:00:00 AM  West 4 St-Washington Sq (A,B,C,D,E,F,M)   
4880458  11/30/2022 08:00:00 PM                            Wall St (4,5)   

                routes   latitude  longitude  1  2  3  4  5  ...  F  J  L  M  \
2      

In [24]:
df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])

/var/folders/6q/9wmlbvpj5nqfn_clctmzyck00000gn/T/ipykernel_4630/3131658029.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])


In [25]:
# Splitting into date and time columns
df['date'] = df['transit_timestamp'].dt.date
df['time'] = df['transit_timestamp'].dt.time

In [26]:
df.head

<bound method NDFrame.head of           transit_timestamp                          station_complex  \
2       2023-01-24 22:00:00                               103 St (6)   
3       2022-09-05 11:00:00                               103 St (6)   
4       2022-05-08 22:00:00                               103 St (1)   
5       2022-05-03 22:00:00                                 1 Av (L)   
6       2023-01-28 18:00:00                                 1 Av (L)   
...                     ...                                      ...   
4880452 2022-10-29 06:00:00  West 4 St-Washington Sq (A,B,C,D,E,F,M)   
4880453 2023-01-27 10:00:00                            Wall St (4,5)   
4880455 2022-07-15 05:00:00                            Wall St (4,5)   
4880456 2023-02-27 06:00:00  West 4 St-Washington Sq (A,B,C,D,E,F,M)   
4880458 2022-11-30 20:00:00                            Wall St (4,5)   

                routes   latitude  longitude  1  2  3  4  5  ...  L  M  N  Q  \
2                    6  4

Similiar things i did to the other data for taxi just to compare them 

In [27]:
# Define a function to assign time slots
def time_slots(x):
    if x.hour in range(6, 12):
        return 'Morning'
    elif x.hour in range(12, 17):
        return 'Afternoon'
    elif x.hour in range(17, 22):
        return 'Evening'
    else:
        return 'Late Night'


In [28]:
df['timeslots'] = df['time'].apply(time_slots)

In [29]:
unique_counts =df['station_complex'].nunique()
# Print the number of unique values for each column
print("Number of unique values for station complex so how many stations do i have ")
print(unique_counts)

Number of unique values for station complex so how many stations do i have 
120


In [30]:
counts =df['routes'].nunique()
# Print the number of unique values for each column
print("Number of routes i have in that area ")
print(counts)

Number of routes i have in that area 
45


In [31]:
df.columns

Index(['transit_timestamp', 'station_complex', 'routes', 'latitude',
       'longitude', '1', '2', '3', '4', '5', '6', '7', 'A', 'B', 'C', 'D', 'E',
       'F', 'J', 'L', 'M', 'N', 'Q', 'R', 'S', 'W', 'Z', 'date', 'time',
       'timeslots'],
      dtype='object')

In [32]:
df.to_csv('mtavisualise.csv', index=False)